In [9]:
%pip install pandas numpy scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
# Load datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Merge datasets to form a comprehensive dataset
merged_data = transactions.merge(customers, on="CustomerID", how="left").merge(products, on="ProductID", how="left")

# Display the merged data structure
print("Merged Data:")
print(merged_data.head())

Merged Data:
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Blue

In [ ]:
# Aggregate transaction and customer-level features
customer_features = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",  # Total spending
    "Quantity": "sum",    # Total quantity purchased
    "Price_x": "mean",      # Average product price
    "Category": lambda x: x.mode()[0] if not x.mode().empty else "Unknown",  # Most purchased category
    "Region": "first"     # Region (from Customers.csv)
}).reset_index()

# One-hot encode categorical columns
customer_features = pd.get_dummies(customer_features, columns=["Category", "Region"], drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
numerical_cols = ["TotalValue", "Quantity", "Price"]
customer_features[numerical_cols] = scaler.fit_transform(customer_features[numerical_cols])

# Display the prepared features
print("Customer Features (after processing):")
print(customer_features.head())

KeyError: "Column(s) ['Price'] do not exist"

In [13]:
# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(customer_features.drop("CustomerID", axis=1))

# Convert similarity matrix to a DataFrame for better handling
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features["CustomerID"], columns=customer_features["CustomerID"])

# Display similarity matrix (for reference)
print("Similarity Matrix:")
print(similarity_df.head())

NameError: name 'customer_features' is not defined

In [14]:
# Get the first 20 CustomerIDs
target_customers = customers.loc[customers["CustomerID"].isin([f"C{str(i).zfill(4)}" for i in range(1, 21)]), "CustomerID"]

# Generate lookalikes for each target customer
lookalike_map = {}
for customer in target_customers:
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]  # Top 3 similar customers (exclude self)
    lookalike_map[customer] = list(zip(similar_customers.index, similar_customers.values))

# Convert lookalike map to a DataFrame
lookalike_data = []
for cust_id, lookalikes in lookalike_map.items():
    for similar_cust, score in lookalikes:
        lookalike_data.append({"cust_id": cust_id, "similar_cust_id": similar_cust, "score": score})

lookalike_df = pd.DataFrame(lookalike_data)

# Save the result to Lookalike.csv
lookalike_df.to_csv("Lookalike.csv", index=False)

# Display the Lookalike DataFrame
print("Lookalike Data:")
print(lookalike_df.head())

NameError: name 'similarity_df' is not defined